# Import Packages

In [ ]:
from PIL import Image, ImageDraw, ImageFont
# from bs4 import BeautifulSoup
import math
import random
import string
import requests
import numpy as np
import math
import os
import cv2
import imageio
import shutil

# Import Word Lists for Text Generation

In [ ]:
# import English word list (10000 entries) for random text generation
with open('/content/drive/MyDrive/Gradient Health/Text Detection Project/Word Lists/English Word List.txt') as f:
    English_list = f.read().split('\n')[:-1]

In [ ]:
# # webscrape Portuguese word list
# page = requests.get('https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Portuguese_wordlist')
# soup = BeautifulSoup(page.text)
# words = soup.find_all('li')
# Portuguese_list = [word.get_text().split(' ')[0]for word in words[:4927]]

# # save to local
# with open('/content/drive/MyDrive/Gradient Health/Text Detection Project/Word Lists/Portuguese Word List.txt', 'w') as f:
#     for word in Portuguese_list:
#         f.write(f'{word}\n')

In [ ]:
# import Portuguese word list (4927 entries) for random text generation
with open('/content/drive/MyDrive/Gradient Health/Text Detection Project/Word Lists/Portuguese Word List.txt') as f:
    Portuguese_list = f.read().split('\n')[:-1]

In [ ]:
# # clean Chinese word list
# with open('/content/drive/MyDrive/Gradient Health/Text Detection Project/Word Lists/Chinese Word List Uncleaned.txt') as source:
#     with open('/content/drive/MyDrive/Gradient Health/Text Detection Project/Word Lists/Chinese Word List.txt', 'w') as dest:
#         for line in source:
#             words = line.split()
#             if words:
#                 dest.write(words[0] + '\n')

In [ ]:
# import Chinese word list (56064 entries) for random text generation
with open('/content/drive/MyDrive/Gradient Health/Text Detection Project/Word Lists/Chinese Word List.txt') as f:
    Chinese_list = f.read().split('\n')[:-1]

# Helper Functions for Random Text Generation

In [ ]:
# generate random text block
# supports English, Portuguese, and Chinese
def generate_random_text(length, language):
    if language == 'English':
        word_list = English_list
        text = ' '.join(random.choice(word_list) for _ in range(length))
        return text
    elif language == 'Portuguese':
        word_list = Portuguese_list
        text = ' '.join(random.choice(word_list) for _ in range(length))
        return text
    elif language == 'Chinese':
        word_list = Chinese_list
        text = ' '.join(random.choice(word_list) for _ in range(length))
        return text
    else:
        return 'Language not supported'

In [ ]:
# generate a random color in RGB format
def generate_random_color():
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return (r, g, b)

In [ ]:
# generate a random grayscale color in RGB format
def generate_random_grayscale_color():
    intensity = random.randint(0, 255)
    return (intensity, intensity, intensity)

In [ ]:
# generate a number from Poisson distribution
def generate_poisson(lower_bound = 1, upper_bound = 10, lambd = 2):
    while True:
        sample = np.random.poisson(lambd)
        if lower_bound <= sample <= upper_bound:
            return sample

# Main Function for Random Text Generation

In [ ]:
# overlay random text on image
# customize randomness
def overlay_random_text(
    input, output,
    language = 'English',
    font_name =  'Arial Unicode.ttf',
    font_size = 20, # pixels
    text_length = 10, # number of characters
    fill_color = (0, 0, 0), # default black
    stroke_color = (255, 255, 255), # default white
    stroke_width = 1, # pixels
    rotation_degree = 0):

    with Image.open(input) as img:

        # makes sure image is in RGB mode
        if img.mode != 'RGB':
            img = img.convert('RGB')

        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype('/content/drive/MyDrive/Gradient Health/Text Detection Project/Fonts/' + font_name, font_size)
        text = generate_random_text(text_length, language)

        # generate random coordinate (top-left)
        x, y = random.randint(0, img.width), random.randint(0, img.height)

        # generate transparent image to temporarily place text
        temp = Image.new('RGBA', (img.width, img.height), (0, 0, 0, 0))  # transparent black background

        # draw text
        temp_draw = ImageDraw.Draw(temp)
        temp_draw.text((x, y), text, font=font, fill=fill_color, stroke_width=stroke_width, stroke_fill=stroke_color)

        # rotate
        temp = temp.rotate(rotation_degree, fillcolor=(0, 0, 0, 0))

        # find the first and last pixels containing text on y axis and x axis
        data = np.array(temp)
        colored = np.any(data[:, :, :] != [0, 0, 0, 0], axis=2)  # find pixels that are not transparent black
        ## y axis
        reduced_y = np.any(colored, axis=1)
        indices_y = np.where(reduced_y)[0]
        first_index_y = indices_y[0] if indices_y.size else 0
        last_index_y = indices_y[-1] if indices_y.size else 0
        ## x axis
        reduced_x = np.any(colored, axis=0)
        indices_x = np.where(reduced_x)[0]
        first_index_x = indices_x[0] if indices_x.size else 0
        last_index_x = indices_x[-1] if indices_x.size else 0

        # # uncomment when checking manual bounding boxes
        # # draw bounding box
        # draw.rectangle((first_index_x, first_index_y, last_index_x, last_index_y), outline = 'red')

        # paste text on to the image
        img.paste(temp, temp)

        # save image
        img.save(output)
        print(f'Image saved in {output}')

        # calculate center_x, center_y, bbox_width, bbox_height
        # then normalize to (0-1)
        center_x = (last_index_x + first_index_x)/2
        center_x_norm = center_x / img.width
        center_y = (last_index_y + first_index_y)/2
        center_y_norm = center_y / img.height
        bbox_width = last_index_x - first_index_x
        bbox_width_norm = bbox_width / img.width
        bbox_height = last_index_y - first_index_y
        bbox_height_norm = bbox_height / img.height

        return center_x_norm, center_y_norm, bbox_width_norm, bbox_height_norm

# Supplemental Functions

In [ ]:
# create a GIF from the images
def create_gif(image_dir, gif_path, duration = 1):
    images = []

    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        images.append(imageio.v2.imread(file_path))

    imageio.mimsave(gif_path, images, duration = duration)

In [ ]:
# create a mp4 video from the images
def create_video(image_dir, video_path, fps=1):
    images = []

    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)
        images.append(cv2.imread(file_path))

    height, width, layers = images[0].shape
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in images:
        video.write(image)

    video.release()

# Generate Images for YOLO

In [ ]:
# %%time
# # select images for modeling
# # copy to a new folder
# source_folder = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs'
# destination_folder = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs for Modeling'

# # ensure the destination directory exists
# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# files = os.listdir(source_folder)

# for file in files:
#     if file.endswith('000.png'):
#         source = os.path.join(source_folder, file)
#         destination = os.path.join(destination_folder, file)

#         # Copy the file
#         shutil.copy(source, destination)

In [ ]:
# %%time
# # crop image to get rid of text-like feature noise
# source_folder = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs for Modeling'
# files = os.listdir(source_folder)

# for file in files:
#     source = os.path.join(source_folder, file)
#     try:
#         img = Image.open(source)
#         width, height = img.size # 1024 by 1024
#         left = 192
#         top = 192
#         right = 832
#         bottom = 832
#         img_cropped = img.crop((left, top, right, bottom)) # 640 by 640
#         img_cropped.save(source)
#     except Exception as e:
#         print(f"Could not process file {file}: {str(e)}")

In [ ]:
# check number of images available
input_path = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs for Modeling'
file_count = sum([len(files) for r, d, files in os.walk(input_path)])
print(f"Total number of files in the directory: {file_count}")

In [ ]:
# # rename images to '1.png', '2.png', ... , '1335.png'
# input_path = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs for Modeling'
# files = [f for f in os.listdir(input_path) if os.path.isfile(os.path.join(input_path, f))]

# for idx, filename in enumerate(files, start=1):
#     old_file_path = os.path.join(input_path, filename)
#     new_file_path = os.path.join(input_path, f"{idx}.png")
#     os.rename(old_file_path, new_file_path)

In [ ]:
# define main function parameters
languages = ['English', 'Portuguese', 'Chinese']
English_Portuguese_font_list = [
    'Arial Unicode.ttf',
    'Geneva.ttf',
    'Keyboard.ttf',
    'Monaco.ttf',
    'NewYork.ttf',
    'NewYorkItalic.ttf',
    'OpenSans-Light.ttf',
    'OpenSans_Condensed-Bold.ttf',
    'OpenSans_Condensed-BoldItalic.ttf',
    'OpenSans_Condensed-ExtraBold.ttf',
    'OpenSans_Condensed-ExtraBoldItalic.ttf',
    'OpenSans_Condensed-Italic.ttf',
    'OpenSans_Condensed-Light.ttf',
    'OpenSans_Condensed-LightItalic.ttf',
    'OpenSans_Condensed-Medium.ttf',
    'OpenSans_Condensed-MediumItalic.ttf',
    'OpenSans_Condensed-Regular.ttf',
    'OpenSans_Condensed-SemiBold.ttf',
    'OpenSans_Condensed-SemiBoldItalic.ttf',
    'OpenSans_SemiCondensed-Bold.ttf',
    'OpenSans_SemiCondensed-BoldItalic.ttf',
    'OpenSans_SemiCondensed-ExtraBold.ttf',
    'OpenSans_SemiCondensed-ExtraBoldItalic.ttf',
    'OpenSans_SemiCondensed-Italic.ttf',
    'OpenSans_SemiCondensed-Light.ttf',
    'OpenSans_SemiCondensed-LightItalic.ttf',
    'OpenSans_SemiCondensed-Medium.ttf',
    'OpenSans_SemiCondensed-MediumItalic.ttf',
    'OpenSans_SemiCondensed-Regular.ttf',
    'OpenSans_SemiCondensed-SemiBold.ttf',
    'OpenSans_SemiCondensed-SemiBoldItalic.ttf',
    'OpenSans-Bold.ttf',
    'OpenSans-BoldItalic.ttf',
    'OpenSans-ExtraBold.ttf',
    'OpenSans-ExtraBoldItalic.ttf',
    'OpenSans-Italic.ttf',
    'OpenSans-LightItalic.ttf',
    'OpenSans-Medium.ttf',
    'OpenSans-MediumItalic.ttf',
    'OpenSans-Regular.ttf',
    'OpenSans-SemiBold.ttf',
    'OpenSans-SemiBoldItalic.ttf',
    'Roboto-ThinItalic.ttf',
    'Roboto-Thin.ttf',
    'Roboto-Regular.ttf',
    'Roboto-MediumItalic.ttf',
    'Roboto-Medium.ttf',
    'Roboto-LightItalic.ttf',
    'Roboto-Light.ttf',
    'Roboto-Italic.ttf',
    'Roboto-BoldItalic.ttf',
    'Roboto-Bold.ttf',
    'Roboto-BlackItalic.ttf',
    'Roboto-Black.ttf',
    'SFCompact.ttf',
    'SFCompactItalic.ttf',
    'SFCompactRounded.ttf',
    'SFNS.ttf',
    'SFNSItalic.ttf',
    'SFNSMono.ttf',
    'SFNSMonoItalic.ttf',
    'SFNSRounded.ttf'
]
Chinese_font_list = [
    'NotoSansSC-Black.otf',
    'NotoSansSC-Bold.otf',
    'NotoSansSC-Light.otf',
    'NotoSansSC-Medium.otf',
    'NotoSansSC-Regular.otf',
    'NotoSansSC-Thin.otf',
    'HanyiSentyPagoda Regular.ttf',
    'HanyiSentyPine Regular.ttf',
    'HanyiSentyWen Regular.ttf'
]
font_size_range = (15,50)
stroke_width_range = (0,1)
rotation_degrees = [
    -2, -1, 0, 1, 2,
    88, 89, 90, 91, 92,
    268, 269, 270, 271, 272
]

## Generate Training Set (80%)

In [ ]:
%%time
# use index 1-1068 images
input_path = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs for Modeling'
for i in range(1, 1069):
    # input
    image_name = str(i) + '.png'
    input = os.path.join(input_path, image_name)
    # output
    output = '/content/drive/MyDrive/Gradient Health/Text Detection Project/custom_dataset/images/train/train ' + str(i) + '.jpg'
    # language
    language = random.choice(languages)
    # font name
    if language == 'Chinese':
        font_name = random.choice(Chinese_font_list)
    else:
        font_name = random.choice(English_Portuguese_font_list)
    # font size
    font_size = random.randint(font_size_range[0], font_size_range[1])
    # text length
    text_length = generate_poisson()
    # fill color
    fill_color = generate_random_grayscale_color()
    # stroke color
    stroke_color = generate_random_grayscale_color()
    # stroke width
    stroke_width = random.randint(stroke_width_range[0], stroke_width_range[1])
    # rotation degree
    rotation_degree = random.choice(rotation_degrees)

    # call main function
    center_x, center_y, bbox_width, bbox_height = overlay_random_text(
        input = input,
        output = output,
        language = language,
        font_name = font_name,
        font_size = font_size,
        text_length = text_length,
        fill_color = fill_color,
        stroke_color = stroke_color,
        stroke_width = stroke_width,
        rotation_degree = rotation_degree)

    # save label to txt file with format (class, center_x, center_y, bbox_width, bbox_height)
    filename =  '/content/drive/MyDrive/Gradient Health/Text Detection Project/custom_dataset/labels/train/train ' + str(i) + '.txt'
    with open(filename, 'w') as file:
        file.write(f'0 {center_x} {center_y} {bbox_width} {bbox_height}')

## Generate Validation Set (20%)

In [ ]:
%%time
# use index 1069-1335 images
input_path = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs for Modeling'
for i in range(1069, 1336):
    # input
    image_name = str(i) + '.png'
    input = os.path.join(input_path, image_name)
    # output
    output = '/content/drive/MyDrive/Gradient Health/Text Detection Project/custom_dataset/images/val/val ' + str(i-1068) + '.jpg'
    # language
    language = random.choice(languages)
    # font name
    if language == 'Chinese':
        font_name = random.choice(Chinese_font_list)
    else:
        font_name = random.choice(English_Portuguese_font_list)
    # font size
    font_size = random.randint(font_size_range[0], font_size_range[1])
    # text length
    text_length = generate_poisson()
    # fill color
    fill_color = generate_random_grayscale_color()
    # stroke color
    stroke_color = generate_random_grayscale_color()
    # stroke width
    stroke_width = random.randint(stroke_width_range[0], stroke_width_range[1])
    # rotation degree
    rotation_degree = random.choice(rotation_degrees)

    # call main function
    center_x, center_y, bbox_width, bbox_height = overlay_random_text(
        input = input,
        output = output,
        language = language,
        font_name = font_name,
        font_size = font_size,
        text_length = text_length,
        fill_color = fill_color,
        stroke_color = stroke_color,
        stroke_width = stroke_width,
        rotation_degree = rotation_degree)

    # save label to txt file with format (class, center_x, center_y, bbox_width, bbox_height)
    filename =  '/content/drive/MyDrive/Gradient Health/Text Detection Project/custom_dataset/labels/val/val ' + str(i-1068) + '.txt'
    with open(filename, 'w') as file:
        file.write(f'0 {center_x} {center_y} {bbox_width} {bbox_height}')

## Generate Testing Set

In [ ]:
%%time
# generate 100 images
input_path = '/content/drive/MyDrive/Gradient Health/Text Detection Project/Radiographs'
input_files = os.listdir(input_path)

for i in range(1,101):
    # input
    random_file = random.choice(input_files)
    input = os.path.join(input_path, random_file)

    # Open the image file
    img = Image.open(input)
    width, height = img.size

    # Ensure the crop box won't be out of the image boundaries
    left = random.randint(0, width - 640)
    upper = random.randint(0, height - 640)
    right = left + 640
    lower = upper + 640

    # Crop the image
    cropped_img = img.crop((left, upper, right, lower))

    # Convert the image to RGB
    rgb_img = cropped_img.convert('RGB')

    # Save the cropped image temporarily
    temp_path = 'temp.jpg'
    rgb_img.save(temp_path)

    # output
    output = '/content/drive/MyDrive/Gradient Health/Text Detection Project/custom_dataset/images/test/test ' + str(i) + '.jpg'
    # language
    language = random.choice(languages)
    # font name
    if language == 'Chinese':
        font_name = random.choice(Chinese_font_list)
    else:
        font_name = random.choice(English_Portuguese_font_list)
    # font size
    font_size = random.randint(font_size_range[0], font_size_range[1])
    # text length
    text_length = generate_poisson()
    # fill color
    fill_color = generate_random_grayscale_color()
    # stroke color
    stroke_color = generate_random_grayscale_color()
    # stroke width
    stroke_width = random.randint(stroke_width_range[0], stroke_width_range[1])
    # rotation degree
    rotation_degree = random.choice(rotation_degrees)

    # call main function
    center_x, center_y, bbox_width, bbox_height = overlay_random_text(
        input = temp_path,
        output = output,
        language = language,
        font_name = font_name,
        font_size = font_size,
        text_length = text_length,
        fill_color = fill_color,
        stroke_color = stroke_color,
        stroke_width = stroke_width,
        rotation_degree = rotation_degree)

    # save label to txt file with format (class, center_x, center_y, bbox_width, bbox_height)
    filename =  '/content/drive/MyDrive/Gradient Health/Text Detection Project/custom_dataset/labels/test/test ' + str(i) + '.txt'
    with open(filename, 'w') as file:
        file.write(f'0 {center_x} {center_y} {bbox_width} {bbox_height}')

    # Remove the temporary image
    os.remove(temp_path)